**0.Set new function** \
The function we discussed in the last 3 tasks, has an issue that its integral value converges to 0. I considered a function whose sum is a positive value. 

In [ ]:
#Test with increasing N, it converges to 0! (100~200)
#WHY??
N = 170

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    term1 = cos(2*π*(j*k/N)) * cos(2*π*(j*l/N))* exp(-j/N)
    term2 = sin(2*π*(j*k/N)) * cos(2*π*(j*l/N))* exp(-j^2/N^2)
    return term1 + term2
end


sum = 0
for j in 0:N
    for k in 0:N
        for l in 0:N
            sum += f((j,k,l)) 
        end
    end
end

println(sum/(N^3))

However, this function also cannot calculate the exact integral value, just like the previous function :(
Because the domain is set to the cube of (x, y, z), not (r, theta). 

**1. Simple Monte Carlo**

In [64]:

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    return (cos(sqrt(j^2+k^2)) * sin(sqrt(j^2+l^2)))^2
end

N = 1000000
sum = 0
for i in 1:N
    v = (rand(0:1),rand(0:1),rand(0:1))
    sum += f(v)
end

sum = sum / N

println(sum)

0.18131589696061196


<img src="Image/newFunctionPositive.png" alt="std" width=800>

It has a value smaller than the actual value. We need Importance sampling!

**2. Monte Carlo with Importance Sampling**

<img src="Image/ImSm.jpg" alt="std" width=800>

Q. How can we ensure that distributions can be extracted using the rand function?
A. https://juliastats.org/Distributions.jl/stable/starting/ (4th code block)

...and truncated function is also helpful dealing with distribution (6th code block)

In [ ]:
using Distributions

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    return (cos(sqrt(j^2 + k^2)) * sin(sqrt(j^2 + l^2)))^2
end

dist = Truncated(Normal(0.0, 0.5), 0.0, 1.0) #Set boundary for distribution (consider domain)

dist3D = Product([dist, dist, dist]) #WOW!

N = 10000000

sum = 0

for _ in 1:N
    randPoint= rand(dist3D) #WOW! - It also possible to visualize distribution (https://discourse.julialang.org/t/product-distribution-allocates-a-lot/126771)
    fv= f(randPoint)
    #pv = 1/(1*1*1)
    qv = pdf(dist3D, randPoint) #get p from distribution
    
    sum += fv / qv
    
end

ISsum = sum / N

println("N : ",N)
println("approx value : ", ISsum)

N : 10000000
approx value : 0.21968970658316425


**3.using monte carlo for j and tci for k,l(high-rank)** \
we do not need to perform TCI on k and l. More specifically, it becomes a Rank 1 matrix. We need a better function to which we can apply TCI!

<img src="Image/mcwithTCI.jpg" alt="std" width=800>

In [ ]:
using Distributions

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    return (cos(sqrt(j^2 + k^2)) * sin(sqrt(j^2 + l^2)))^2
end

N = 100000

sum = 0

for _ in 0:N
    randJ= rand(0:1) 
    
    M = 100
    sumK = 0
    sumL = 0
    for i in 1:M
        sumK += f((randJ,i/M,0))
        sumL += f((randJ,0,i/M))
    end

    fv = sumK * sumL / M^2
    
    # I have to consider how to make my own distribution :(
    
end


**Problem**

I think the key point in importance sampling is setting the distribution correctly.
this time, large values were clustered near zero, so applying standard deviation worked well **by chance** as I think :(.